In [1]:
import os
import yaml
from torch.utils.data import DataLoader
import argparse

from GeospatialFM.data import get_datasets
from GeospatialFM.models import *
# from utils import load_config
from torchgeo.samplers import RandomGeoSampler
from matplotlib import pyplot as plt

from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup
from GeospatialFM.utils import setup, get_eval_fn, get_data, init_distributed_device
from GeospatialFM.data import *
from GeospatialFM.models import *
from GeospatialFM.loss import *

from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import numpy as np
from torch.utils.data import ConcatDataset
import segmentation_models_pytorch as smp

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:0')

In [3]:
args = {'exp_name': None,
        'config_file': 'GeospatialFM/configs/pretrain_cvit.yaml',
        'opts': None, 
        'save_config': False}
args = argparse.Namespace(**args)
args.debug = True
args.finetune = False
args

Namespace(exp_name=None, config_file='GeospatialFM/configs/pretrain_cvit.yaml', opts=None, save_config=False, debug=True, finetune=False)

In [24]:
device = init_distributed_device(args)
cfg, _ = setup(args)

AttributeError: 'Namespace' object has no attribute 'rank'

In [ ]:
cfg.DATASET['train_transforms']['normalize'] = True
cfg.DATASET['train_transforms']['standardize'] = False

In [ ]:
data = get_data(cfg)
data

In [ ]:
sample = iter(data['test'].dataloader).__next__()

In [8]:
sample['image'].shape

In [9]:
model = construct_mae(cfg.MODEL)

In [10]:
device = 'cuda:0'
device_ids = [0, 1, 2, 3]

In [11]:
model = model.to(device)
model = torch.nn.DataParallel(model, device_ids=device_ids)

In [12]:
loss = get_loss_list(cfg.LOSS)

In [13]:
loss

In [ ]:
device = torch.device(device)

In [ ]:
# image = sample['image'].to(device, non_blocking=True)
# radar = sample['radar'].to(device, non_blocking=True)
image = sample['image']
radar = sample['radar']

In [ ]:
# model_out = model(image, radar)
# logit_scale = model_out.get("logit_scale").mean()
# model_out['logit_scale'] = logit_scale

In [ ]:
# points = [[20, 193], [35, 180], [55, 150], [130, 180], [95, 140], [30, 25], [80, 30]]
points = [[45, 10], [45, 20]]
cross_length = 5
img_id = 100
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
color = ['r', 'g', 'b', 'cyan', 'orange', 'purple', 'brown']
for c, point in enumerate(points):
    curve = image[img_id, :, point[0], point[1]].detach().cpu().numpy()
    curve = np.delete(curve, 10)
    plt.plot(np.arange(12), curve, color[c])
vis_img = image[img_id, [3, 2, 1], :, :].detach().cpu().numpy().transpose(1, 2, 0)
# normalize each channel
vis_img = (vis_img - vis_img.min()) / (vis_img.max() - vis_img.min())

plt.subplot(1, 2, 2)
plt.imshow(vis_img)
# draw a cross at the point
for c, point in enumerate(points):
    plt.plot([point[1] - cross_length, point[1] + cross_length], [point[0], point[0]],  color[c]) 
    # Draw vertical line of the cross
    plt.plot([point[1], point[1]], [point[0] - cross_length, point[0] + cross_length],  color[c]) 
# plt.axis('off')

In [ ]:
# points = [[20, 193], [35, 180], [55, 150], [130, 180], [95, 140], [30, 25], [80, 30]]
points = [[45, 10], [45, 20]]
cross_length = 5
img_id = 150
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
color = ['r', 'g', 'b', 'cyan', 'orange', 'purple', 'brown']
for c, point in enumerate(points):
    curve = radar[img_id, :, point[0], point[1]].detach().cpu().numpy()
    # curve = np.delete(curve, 10)
    plt.plot(np.arange(2), curve, color[c])
vis_img = radar[img_id, [0], :, :].detach().cpu().numpy().transpose(1, 2, 0)
# normalize each channel
vis_img = (vis_img - vis_img.min()) / (vis_img.max() - vis_img.min())

plt.subplot(1, 2, 2)
plt.imshow(vis_img)
# draw a cross at the point
for c, point in enumerate(points):
    plt.plot([point[1] - cross_length, point[1] + cross_length], [point[0], point[0]],  color[c]) 
    # Draw vertical line of the cross
    plt.plot([point[1], point[1]], [point[0] - cross_length, point[0] + cross_length],  color[c]) 
# plt.axis('off')

In [1]:
from GeospatialFM.models.costum_layers import *
import torch

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
sample = torch.randn(2, 13, 224, 224)

In [53]:
patch_embedding = PatchEmbedPerChannel(in_chans=13)

In [54]:
extra_tokens = dict()
extra_tokens["channels"] = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]])
out = patch_embedding(sample, extra_tokens)

In [57]:
collpase_channels(out[0], 'mean').shape

torch.Size([2, 768])

In [61]:
out[1].shape

torch.Size([2, 13])

In [ ]:
out[1]

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [35]:
13*0.25

3.25

In [21]:
x = sample
B, Cin, H, W = x.shape
# Note: The current number of channels (Cin) can be smaller or equal to in_chans
len_keep = int(Cin * 0.4)

noise = torch.rand(B, Cin, device=x.device)  # noise in [0, 1]
# sort noise for each sample
ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
ids_restore = torch.argsort(ids_shuffle, dim=1)

# keep the first subset
ids_keep = ids_shuffle[:, :len_keep]

In [68]:
ids_keep.unsqueeze(-1).unsqueeze(-1).repeat(1, 5, 224, 224)

tensor([[[[ 5,  5,  5,  ...,  5,  5,  5],
          [ 5,  5,  5,  ...,  5,  5,  5],
          [ 5,  5,  5,  ...,  5,  5,  5],
          ...,
          [ 5,  5,  5,  ...,  5,  5,  5],
          [ 5,  5,  5,  ...,  5,  5,  5],
          [ 5,  5,  5,  ...,  5,  5,  5]],

         [[ 1,  1,  1,  ...,  1,  1,  1],
          [ 1,  1,  1,  ...,  1,  1,  1],
          [ 1,  1,  1,  ...,  1,  1,  1],
          ...,
          [ 1,  1,  1,  ...,  1,  1,  1],
          [ 1,  1,  1,  ...,  1,  1,  1],
          [ 1,  1,  1,  ...,  1,  1,  1]],

         [[ 6,  6,  6,  ...,  6,  6,  6],
          [ 6,  6,  6,  ...,  6,  6,  6],
          [ 6,  6,  6,  ...,  6,  6,  6],
          ...,
          [ 6,  6,  6,  ...,  6,  6,  6],
          [ 6,  6,  6,  ...,  6,  6,  6],
          [ 6,  6,  6,  ...,  6,  6,  6]],

         ...,

         [[ 6,  6,  6,  ...,  6,  6,  6],
          [ 6,  6,  6,  ...,  6,  6,  6],
          [ 6,  6,  6,  ...,  6,  6,  6],
          ...,
          [ 6,  6,  6,  ...,  6,  6, 

In [52]:
ids_keep.shape
# sample corresponding channels from ce
new_ce = torch.gather(ce, 2, ids_keep.unsqueeze(1).repeat(1, ce.shape[1], 1))

In [50]:
new_ce[0, :5, 0], ids_keep[0, 0], ce[0, :5, 2]

(tensor([ 0.0149,  0.0129,  0.0069, -0.0358,  0.0500],
        grad_fn=<SelectBackward0>),
 tensor(2),
 tensor([ 0.0149,  0.0129,  0.0069, -0.0358,  0.0500],
        grad_fn=<SelectBackward0>))

In [46]:
new_ce[0, :5].shape

torch.Size([5, 5])

In [49]:
ce[0, :5, 2].shape

torch.Size([5])

In [77]:
# gather the C dim of the batch of the images in shape B x C x H x W following the index of ids_keep
x_keep = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).unsqueeze(-1).repeat(1, 1, *x.shape[-2:]))

In [78]:
x_keep.shape

torch.Size([2, 5, 224, 224])

In [79]:
# generate the binary mask: 0 is keep, 1 is remove
mask = torch.ones([B, Cin], device=x.device)
mask[:, :len_keep] = 0
# unshuffle to get the binary mask
mask = torch.gather(mask, dim=1, index=ids_restore)
# Per batch channel sampling
        # Note this may be slow

In [80]:
mask

tensor([[0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0.],
        [1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0.]])

In [81]:
ids_keep

tensor([[ 5,  1,  6, 12,  0],
        [ 2, 11,  1, 12,  8]])

In [2]:
embedding = torch.rand((2, 3, 10))

In [4]:
embedding.shape

torch.Size([2, 3, 10])

In [5]:
in_chans = 13
embed_dim = 768
patch_size = 16

proj = nn.Conv2d(
    in_chans,
    embed_dim*in_chans,
    kernel_size=(patch_size, patch_size),
    stride=(patch_size, patch_size),
    groups=in_chans,
)  # CHANGED

In [9]:
proj2 = nn.Conv2d(
    in_chans,
    embed_dim,
    kernel_size=(patch_size, patch_size),
    stride=(patch_size, patch_size),
)  # CHANGED

In [11]:
# print the number of parameters
w1 = sum(p.numel() for p in proj.parameters())
w2 = sum(p.numel() for p in proj2.parameters())
w1-w2

9216

In [8]:
proj(sample).view(-1, in_chans, embed_dim, 14, 14).shape

torch.Size([2, 13, 768, 14, 14])

In [12]:
sample.shape

torch.Size([2, 13, 224, 224])

In [14]:
sample[:, torch.tensor([0, 1, 11])].shape

torch.Size([2, 3, 224, 224])